In [1]:
from matplotlib import pyplot as plt
from utils.utils import *
import utils.promethee_functions as pf
import utils.clustering_functions as cf

data = read_data()

group0 = ["PAK", "SDN", "BDI", "HTI"]
group1 = ["EST", "CZE", "MLT", "SGP", "IRL"]
group2 = ["CHE", "ISL", "NZL", "SWE"]

all_groups = group0 + group1 + group2

data = data.loc[all_groups]

data = scale_data(data)

print("\nData scaled")
get_min_max_criteria(data, False)

L = data.iloc[0]["co2prod"].shape[0] # Length of the time series

utils.py Loading
Reading HDI dataset
co2prod: min=0.0, max=33.3863
hdi: min=0.257, max=0.967
le: min=37.105, max=85.473
gdi: min=0.383, max=1.041
eys: min=3.5751, max=23.2477
mys: min=1.4606, max=14.2559

Data scaled
co2prod: min=0.0, max=1.0
hdi: min=0.0, max=1.0
le: min=0.0, max=1.0
gdi: min=0.0, max=1.0
eys: min=0.0, max=1.0
mys: min=0.0, max=1.0


In [22]:
n_samples = data.shape[0]
n_features = data.shape[1]

formatted_data = np.stack([np.stack(data.iloc[:, i].values) for i in range(n_features)], axis=-1)

formatted_data.shape

names = data.index
names_formatted = [name for name in names]


In [20]:
formatted_data

array([[[0.97580131, 0.19295775, 0.48697773, 0.23708207, 0.04096811,
         0.06680378],
        [0.97672191, 0.2       , 0.49107105, 0.28419453, 0.0435882 ,
         0.07472036],
        [0.97576772, 0.2056338 , 0.48803818, 0.29331307, 0.04620829,
         0.08263695],
        ...,
        [0.96322968, 0.39295775, 0.61853659, 0.68085106, 0.26689367,
         0.2380146 ],
        [0.95970884, 0.3943662 , 0.61490986, 0.68541033, 0.26689367,
         0.2380146 ],
        [0.95970884, 0.39859155, 0.62197243, 0.68541033, 0.26689367,
         0.2380146 ]],

       [[0.99119653, 0.0915493 , 0.26780488, 0.56534954, 0.        ,
         0.00316712],
        [0.99195951, 0.1       , 0.27800636, 0.57294833, 0.00873223,
         0.01072128],
        [0.99302522, 0.08591549, 0.18634146, 0.62613982, 0.01780968,
         0.01827544],
        ...,
        [0.98115669, 0.36760563, 0.60464475, 0.73708207, 0.30436881,
         0.19362837],
        [0.9801348 , 0.36478873, 0.59728526, 0.73860182, 0.304

In [25]:
from tslearn.clustering import TimeSeriesKMeans

n_clusters = 3

km = TimeSeriesKMeans(n_clusters=n_clusters, metric="euclidean", max_iter=5).fit(formatted_data)
km.labels_

array([1, 1, 1, 1, 0, 0, 0, 0, 0, 2, 2, 2, 2])

In [39]:
# Compute the dunn index

# Dunn index = min_{1<=i!=j<=k} (d(C_i, C_j) / max_{1<=l<=k} d(C_l))
# where d(C_i, C_j) is the distance between clusters C_i and C_j

# Compute the distance between clusters
distances = []
for i in range(n_clusters):
    dist = 0
    for j in range(i + 1, n_clusters):
        cluster1 = formatted_data[km.labels_ == i]
        cluster2 = formatted_data[km.labels_ == j]
        dist += np.linalg.norm(cluster1.mean(axis=0) - cluster2.mean(axis=0))
    distances.append(dist)
distances = np.array(distances)

# Compute the maximum distance between clusters
max_distance = np.max(distances)


# Compute the intra-cluster distance
intra_cluster_distances = np.zeros(n_clusters)
for i in range(n_clusters):
    cluster = formatted_data[km.labels_ == i]
    distances = np.zeros((cluster.shape[0], cluster.shape[0]))
    for j in range(cluster.shape[0]):
        for k in range(cluster.shape[0]):
            distances[j, k] = np.linalg.norm(cluster[j] - cluster[k])
    intra_cluster_distances[i] = np.mean(distances)

# Compute the Dunn index
dunn_index = np.min(distances) / np.max(intra_cluster_distances)

dunn_index

np.float64(0.0)

In [40]:
distances

array([[0.        , 1.49357997, 1.28684379, 0.97289973],
       [1.49357997, 0.        , 0.70408309, 1.36664567],
       [1.28684379, 0.70408309, 0.        , 0.9923812 ],
       [0.97289973, 1.36664567, 0.9923812 , 0.        ]])

In [35]:
intra_cluster_distances

array([1.29888477, 1.53674178, 0.85205418])

In [28]:
def dunn_index(km, formatted_data):
    clusters = [formatted_data[km.labels_ == i] for i in range(km.n_clusters)]
    print(clusters)
    
    intra_cluster_distances = [np.mean([np.linalg.norm(a - b) for a in cluster for b in cluster]) for cluster in clusters]

    inter_cluster_distances = [np.mean([np.linalg.norm(a - b) for a in cluster1 for b in cluster2]) for cluster1 in clusters for cluster2 in clusters if cluster1 != cluster2]

    return min(inter_cluster_distances) / max(intra_cluster_distances)

dunn_index(km, formatted_data)


[array([[[0.        , 0.68169014, 0.69408271, 0.94680851, 0.58722099,
         0.65857343],
        [0.07217076, 0.67464789, 0.69155885, 0.94680851, 0.56873854,
         0.67824309],
        [0.32579555, 0.66056338, 0.68375398, 0.95136778, 0.55885109,
         0.69791275],
        [0.45067504, 0.64929577, 0.66752916, 0.95592705, 0.54076158,
         0.71758241],
        [0.42247632, 0.65211268, 0.65281018, 0.97568389, 0.55416437,
         0.73725207],
        [0.47148067, 0.67042254, 0.67064687, 0.97568389, 0.56972644,
         0.75692173],
        [0.43566269, 0.69295775, 0.70074231, 0.96808511, 0.59171558,
         0.77775823],
        [0.44121167, 0.71408451, 0.71319194, 0.97112462, 0.61689418,
         0.79859474],
        [0.48967691, 0.72535211, 0.70226935, 0.97720365, 0.64207279,
         0.81943124],
        [0.51731025, 0.73802817, 0.72131495, 0.9787234 , 0.65738521,
         0.84026774],
        [0.52892693, 0.76197183, 0.72587487, 0.98328267, 0.71589506,
         0.86110425]

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [26]:
# Give the name of each elem and in which cluster it is
for i in range(n_samples):
    print(names_formatted[i], km.labels_[i])

# Get the centroids
centroids = km.cluster_centers_

# Get the closest element to each centroid
closest_elements = []

for i in range(n_clusters):
    closest_elements.append(np.argmin([np.linalg.norm(centroids[i] - formatted_data[j]) for j in range(n_samples)]))

closest_elements

# Get the name of the closest element to each centroid
for i in range(n_clusters):
    print(names_formatted[closest_elements[i]])

# Get the distance between each centroid
for i in range(n_clusters):
    for j in range(i+1, n_clusters):
        print("Distance between cluster", i, "and cluster", j, ":", np.linalg.norm(centroids[i] - centroids[j]))

# Get the distance between each element and each centroid
for i in range(n_samples):
    for j in range(n_clusters):
        print("Distance between", names_formatted[i], "and cluster", j, ":", np.linalg.norm(formatted_data[i] - centroids[j]))



PAK 1
SDN 1
BDI 1
HTI 1
EST 0
CZE 0
MLT 0
SGP 0
IRL 0
CHE 2
ISL 2
NZL 2
SWE 2
CZE
SDN
NZL
Distance between cluster 0 and cluster 1 : 6.647528971872454
Distance between cluster 0 and cluster 2 : 1.3384972933589878
Distance between cluster 1 and cluster 2 : 7.395459244180558
Distance between PAK and cluster 0 : 6.851583911808431
Distance between PAK and cluster 1 : 1.6388695225946812
Distance between PAK and cluster 2 : 7.561364188351706
Distance between SDN and cluster 0 : 6.881615019781814
Distance between SDN and cluster 1 : 0.5764594731676381
Distance between SDN and cluster 2 : 7.649882106768653
Distance between BDI and cluster 0 : 7.503528128959126
Distance between BDI and cluster 1 : 1.3945165179948458
Distance between BDI and cluster 2 : 8.281579813827769
Distance between HTI and cluster 0 : 5.72763684146373
Distance between HTI and cluster 1 : 1.2992862148205528
Distance between HTI and cluster 2 : 6.414265406379273
Distance between EST and cluster 0 : 1.4309253392983314
Distanc